In [1]:
import wikipedia 
import transformers
from pathlib import Path 
from transformers import AutoModelForCausalLM, AutoTokenizer
transformers.utils.logging.set_verbosity(transformers.logging.CRITICAL)



# My personal cache directory
cache_dir = Path('/n/netscratch/vadhan_lab/Lab/rrinberg/HF_cache')
data_cache= Path("/n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia")

if not cache_dir.exists():
    cache_dir = None 
if not data_cache.exists():
    data_cache = None 
    

import os
os.environ["HF_HOME"] = str(cache_dir)


os.environ["TRANSFORMERS_CACHE"] = str(cache_dir)
os.environ["HF_DATASETS_CACHE"] = str(cache_dir)
os.environ["HF_HUB_CACHE"] = str(cache_dir)

In [2]:
import torch 
from transformers.utils import logging
logging.set_verbosity_debug()

model_id = 'HuggingFaceH4/zephyr-7b-beta'

#model_id = "mistralai/Mistral-7B-Instruct-v0.3"
print(f"model_id- {model_id}")
print(f"cache_dir - {cache_dir}")   
device = 'cuda:0'
dtype= torch.float32
model = AutoModelForCausalLM.from_pretrained(model_id,
                 
                                             torch_dtype=dtype, cache_dir=cache_dir,)
model = model.to(device)
model.requires_grad_(False)
tokenizer = AutoTokenizer.from_pretrained(model_id, 
                                          use_fast=False, cache_dir=cache_dir)

model_id- HuggingFaceH4/zephyr-7b-beta
cache_dir - /n/netscratch/vadhan_lab/Lab/rrinberg/HF_cache


loading configuration file config.json from cache at /n/netscratch/vadhan_lab/Lab/rrinberg/HF_cache/models--HuggingFaceH4--zephyr-7b-beta/snapshots/892b3d7a7b1cf10c7a701c60881cd93df615734c/config.json
Model config MistralConfig {
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 2,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.1",
  "use_cache": true,
  "vocab_size": 32000
}

Detected torch version: 2.5.1
Detected torch version: 2.5.1
loading weights file model.safetensors from cache at /n/netscratch/vadhan_lab/

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing MistralForCausalLM.

All the weights of MistralForCausalLM were initialized from the model checkpoint at HuggingFaceH4/zephyr-7b-beta.
If your task is similar to the task the model of the checkpoint was trained on, you can already use MistralForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /n/netscratch/vadhan_lab/Lab/rrinberg/HF_cache/models--HuggingFaceH4--zephyr-7b-beta/snapshots/892b3d7a7b1cf10c7a701c60881cd93df615734c/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2
}

loading file tokenizer.model from cache at /n/netscratch/vadhan_lab/Lab/rrinberg/HF_cache/models--HuggingFaceH4--zephyr-7b-beta/snapshots/892b3d7a7b1cf10c7a701c60881cd93df615734c/tokenizer.model
loading file added_tokens.json from cache at /n/netscratch/vadhan_lab/Lab/rrinberg/HF_cache/models--HuggingFaceH4--zephyr-7b-beta/snapshots/892b3d7a7b

In [3]:
from importlib import reload 

from wiki_rag import wikipedia
reload(wikipedia)
from wiki_rag import rag

import os

from pathlib import Path
data_cache= Path("/n/netscratch/vadhan_lab/Lab/rrinberg/wikipedia/text")





for i, article in enumerate(wikipedia.parse_wikiextractor_output(data_cache)):
    print(article['title'])
    print(article.keys())
    
    text = article['text']
    if len(text) < 100:
        continue
    
    # remove any leading or trailing whitespace
    text = text.strip()
    print(len(text))
    print(text[:1000])  # preview first 300 characters
    if i >10:
        break  # remove this to keep going


Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-forme

Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-forme

In [4]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.embeddings.base import Embeddings
from typing import List
import numpy as np
from pathlib import Path 
import json
import os
from wiki_rag import wikipedia
from itertools import islice
from typing import Iterator


# === Helper to batch an iterator ===
def batched(iterable: Iterator, batch_size: int):
    iterator = iter(iterable)
    while batch := list(islice(iterator, batch_size)):
        yield batch

embeddings = rag.ModelEmbeddings(model, tokenizer, device)
vectorstore = None

wiki_generator = wikipedia.parse_wikiextractor_output(data_cache)
counts = 0
batch_size = 4

docs = []
    
for i, d in enumerate(wiki_generator):
    #print(f"Processing article {counts}: {d['title']}")
    if counts > int(10):
        break

    title = d['title']
    url = d['url']
    text = d['text']
    id_ = d.get('id')
    if len(text) < 100:
        continue
    counts +=1
    
    if counts % 25 == 0:
        print(f"Processed {counts} articles so far...")
    
    text = text.strip()
    # abstract is first 3 par
    abstract = "\n".join(text.split("\n")[:3])
    doc = Document(page_content=abstract, metadata={"title": title})
    
    if vectorstore is None:
        vectorstore = FAISS.from_documents([doc], embeddings)
    else:
        vectorstore.add_documents([doc])

index_path = "wiki_faiss_index"
if vectorstore:
    vectorstore.save_local(index_path)
    print(f"✅ FAISS index saved to {index_path}")
else:
    print("⚠️ No documents were indexed.")


Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-forme

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-formed (invalid token): line 1, column 0
Skipping malformed doc: not well-forme

In [10]:
resp = vectorstore.search("what is the Hellenic parliament?", k=10, search_type="similarity")
resp 
#num_elements = 
#vectorstore.index.ntotal


[Document(id='84a96fe5-aeae-47b3-9a57-9b32a4b9702f', metadata={'title': 'Thessaloniki B'}, page_content='Thessaloniki B\n\nThessaloniki Beta () is a constituency of the Hellenic Parliament. It comprises Thessaloniki Prefecture except for the urban area of Thessaloniki, which constitutes Thessaloniki A. It elects nine members of parliament.'),
 Document(id='23b58f45-7556-4bff-95d6-76e3b31a1c47', metadata={'title': 'Nardus van der Walt'}, page_content='Nardus van der Walt\n\nHendrik Bernardus van der Walt (born ) is a South African professional rugby union player for in the Rugby Pro D2 in France. His regular position is flanker or number eight.'),
 Document(id='8e9e14f5-62e4-46eb-a680-ee0c677e3958', metadata={'title': 'Portrait of a Young Man with a Book (Bronzino)'}, page_content='Portrait of a Young Man with a Book (Bronzino)\n\nThe Portrait of a Young Man with a Book is a painting by Agnolo Bronzino created in the 1530s. After its creation, it was owned amongst various aristocrats an

In [15]:
[a for a in dir(resp[0]) if "__" not in a]
resp[0].metadata

{'title': 'Thessaloniki B'}

In [6]:

wiki_generator = wikipedia.wikipedia_abstract_generator(data_cache)

for _ in range(3):
    print(next(wiki_generator))

('Anarchism', 'Anarchism is a political philosophy and movement that seeks to abolish all institutions that perpetuate authority, coercion, or hierarchy, primarily targeting the state and capitalism. Anarchism advocates for the replacement of the state with stateless societies and voluntary free associations. A historically left-wing movement, anarchism is usually described as the libertarian wing of the socialist movement (libertarian socialism).')
('Albedo', 'Albedo ( ; ) is the fraction of sunlight that is diffusely reflected by a body. It is measured on a scale from 0 (corresponding to a black body that absorbs all incident radiation) to 1 (corresponding to a body that reflects all incident radiation). "Surface albedo" is defined as the ratio of radiosity "J"e to the irradiance "E"e (flux per unit area) received by a surface. The proportion reflected is not only determined by properties of the surface itself, but also by the spectral and angular distribution of solar radiation reac